In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from collections import Counter

# === 1. เตรียมข้อมูล Iris ===
X, y = load_iris(return_X_y=True)
X_real = X[y == 0][:50]  # Setosa only
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_real)

# === 2. สร้าง QCBM circuit บน Qiskit (4 Qubits)
def create_qcbm(params):
    qc = QuantumCircuit(4)
    for i in range(4):
        qc.ry(params[i], i)
    for i in range(3):  # 0->1, 1->2, 2->3
        qc.cx(i, i+1)
    for i in range(4):
        qc.rz(params[i + 4], i)
    qc.measure_all()
    return qc

# === 3. สุ่ม bitstring จากวงจร
def sample_bitstrings(qc, shots=1024):
    backend = Aer.get_backend("qasm_simulator")
    compiled = transpile(qc, backend)
    job = backend.run(compiled, shots=shots)
    result = job.result()
    counts = result.get_counts()
    bitstrings = []
    for b, c in counts.items():
        bitstrings += [b[::-1]] * c  # Qiskit: LSB first, flip
    return bitstrings

# === 4. แปลง bitstring → scaled feature → inverse scale
def bit_to_scaled(bs):
    return [0.25 if b == '0' else 0.75 for b in bs]

def decode_bitstrings_qiskit(bitstrings, scaler):
    scaled_data = np.array([bit_to_scaled(bs) for bs in bitstrings])
    return scaler.inverse_transform(scaled_data)

# === 5. เรียกใช้งาน
np.random.seed(42)
initial_params = np.random.uniform(0, np.pi, 8)
qc = create_qcbm(initial_params)
bitstrings = sample_bitstrings(qc, shots=300)  # สุ่ม 300 ตัวอย่าง
decoded_data = decode_bitstrings_qiskit(bitstrings, scaler)

# === 6. บันทึกผลลัพธ์เป็น DataFrame
df = pd.DataFrame(decoded_data, columns=["sepal length", "sepal width", "petal length", "petal width"])
df["label"] = "Setosa (QCBM-Qiskit)"

# แสดงบางส่วน
print(df.head())

# (ทางเลือก) บันทึกไฟล์
df.to_csv("qcbm_qiskit_iris_synthetic.csv", index=False)


   sepal length  sepal width  petal length  petal width                 label
0         5.425        2.825         1.225        0.225  Setosa (QCBM-Qiskit)
1         5.425        2.825         1.225        0.225  Setosa (QCBM-Qiskit)
2         5.425        2.825         1.225        0.225  Setosa (QCBM-Qiskit)
3         5.425        2.825         1.225        0.225  Setosa (QCBM-Qiskit)
4         4.675        2.825         1.675        0.475  Setosa (QCBM-Qiskit)


In [9]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
# from qiskit.utils import algorithm_globals
from scipy.optimize import minimize
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import pandas as pd

# === 1. เตรียมข้อมูล Iris และ Scaler ===
X, y = load_iris(return_X_y=True)
X = X[y == 0][:50]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# === 2. แปลงข้อมูลเป็น 4-bit strings (threshold > 0.5)
def real_to_bit(x):
    return ''.join(['1' if val > 0.5 else '0' for val in x])

bitstrings = [real_to_bit(x) for x in X_scaled]

def target_distribution(bitstrings):
    counts = Counter(bitstrings)
    total = sum(counts.values())
    dist = np.zeros(16)
    for bs, c in counts.items():
        idx = int(bs, 2)
        dist[idx] = c / total
    return dist

target_probs = target_distribution(bitstrings)

# === 3. QCBM circuit (parametrized) ===
def create_qcbm(params):
    qc = QuantumCircuit(4)
    for i in range(4):
        qc.ry(params[i], i)
    for i in range(3):  # entangle
        qc.cx(i, i + 1)
    for i in range(4):
        qc.rz(params[i + 4], i)
    qc.measure_all()
    return qc

# === 4. วัด distribution จาก QCBM
def qcbm_distribution(params, shots=1000):
    qc = create_qcbm(params)
    backend = Aer.get_backend("qasm_simulator")
    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    probs = np.zeros(16)
    for b, c in counts.items():
        b = b[::-1]  # flip LSB
        idx = int(b, 2)
        probs[idx] = c / shots
    return probs

# === 5. Loss function: MSE ระหว่าง QCBM กับ Target
def loss(params):
    probs = qcbm_distribution(params)
    return np.sum((probs - target_probs)**2)

# === 6. เริ่มเทรนด้วย scipy.optimize
np.random.seed(42)
initial_params = np.random.uniform(0, np.pi, 8)
result = minimize(loss, initial_params, method="COBYLA", options={"maxiter": 100, "disp": True})
trained_params = result.x

# === 7. สุ่มข้อมูลจากวงจรที่เทรนแล้ว
def sample_qcbm(params, n=300):
    qc = create_qcbm(params)
    backend = Aer.get_backend("qasm_simulator")
    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=n)
    result = job.result()
    counts = result.get_counts()
    samples = []
    for b, c in counts.items():
        b = b[::-1]
        samples += [b] * c
    return samples

bitstrings_generated = sample_qcbm(trained_params, n=1024)

# === 8. Decode bitstring → Scaled features → Original Iris features
def bit_to_scaled(bs):
    return [0.25 if b == '0' else 0.75 for b in bs]

def decode_bitstrings_to_iris(bitstrings, scaler):
    scaled_data = np.array([bit_to_scaled(bs) for bs in bitstrings])
    return scaler.inverse_transform(scaled_data)

decoded_data = decode_bitstrings_to_iris(bitstrings_generated, scaler)

# === 9. แสดงผลลัพธ์
df = pd.DataFrame(decoded_data, columns=["sepal length", "sepal width", "petal length", "petal width"])
df["label"] = "Setosa (QCBM-trained)"

print(df.head())

# (บันทึกไว้ใช้ต่อ)
df.to_csv("qcbm_trained_iris.csv", index=False)



   Normal return from subroutine COBYLA

   NFVALS =   90   F = 6.478200E-02    MAXCV = 0.000000E+00
   X = 1.505245E+00   5.735209E+00   1.245531E+00   1.619065E+00   1.867389E+00
       2.469111E-01   1.647461E+00   2.318846E+00
   sepal length  sepal width  petal length  petal width                  label
0         5.425        2.825         1.675        0.475  Setosa (QCBM-trained)
1         5.425        2.825         1.675        0.475  Setosa (QCBM-trained)
2         4.675        3.875         1.675        0.475  Setosa (QCBM-trained)
3         4.675        3.875         1.675        0.475  Setosa (QCBM-trained)
4         4.675        3.875         1.675        0.475  Setosa (QCBM-trained)
